In [1]:
import ee
import geemap
import numpy as np
import pandas as pd
import copy
import math
import sys,os
sys.path.append(os.path.dirname(os.path.abspath('__file__')))
from tqdm import trange,tqdm
from Basic_tools import *
from New_Correct import *
from Correct_filter import *
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

geemap version = 0.20.3
geemap path = ['D:\\Code_base\\anaconda\\envs\\GEE\\lib\\site-packages\\geemap']


In [3]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [19]:
START_DATE  = ee.Date('2019-08-01')
END_DATE   = ee.Date('2019-08-30')

DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)

models = ['volume', 'surface', None]
Buffer = 0
Model = models[0]
I = 1143
scale = 30

AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(I))
s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filterBounds(AOI)
            .filterDate(START_DATE, END_DATE))
AOI_buffer = AOI.buffer(distance=10000).bounds()
volumetric_dict = {}

In [23]:
s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

image = s1_descending
orbitProperties_pass = image.get('orbitProperties_pass').getInfo()
# get the image geometry and projection
geom = image.geometry()
proj = image.select(1).projection()

def getASCCorners(image, AOI_buffer):
    # 真实方位角
    coords = ee.Array(image.geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))

    if orbitProperties_pass == 'ASCENDING':
        azimuth = azimuth.add(270.0)
        rotationFromNorth = azimuth.subtract(360.0)
    elif orbitProperties_pass == 'DESCENDING':
        azimuth = azimuth.add(180.0)
        rotationFromNorth = azimuth.subtract(180.0)
    else:
        raise TypeError
    azimuthEdge = (ee.Feature(ee.Geometry.LineString([crdLons.get(crdLats.indexOf(minLat)), minLat, minLon,
                              crdLats.get(crdLons.indexOf(minLon))]), {'azimuth': azimuth}).copyProperties(image))

    coords = ee.Array(image.clip(
        AOI_buffer).geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))
    if orbitProperties_pass == 'ASCENDING':
        # 左上角
        startpoint = ee.List([minLon, maxLat])
        # 右下角
        endpoint = ee.List([maxLon, minLat])
    elif orbitProperties_pass == 'DESCENDING':
        # 右上角
        startpoint = ee.List([maxLon, maxLat])
        # 左下角
        endpoint = ee.List([minLon, minLat])

    coordinates_dict = {'crdLons': crdLons, 'crdLats': crdLats,
                        'minLon': minLon, 'maxLon': maxLon, 'minLat': minLat, 'maxLat': maxLat}

    return azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict

Angle_aspect = ee.Terrain.aspect(image.select('angle'))
# s1_azimuth_across = Angle_aspect.reduceRegion(ee.Reducer.mean(), geom, 1000).get('aspect')
azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict = getASCCorners(image, AOI_buffer)
Heading = azimuthEdge.get('azimuth')
Heading_Rad = ee.Image.constant(Heading).multiply(np.pi / 180)

s1_azimuth_across = ee.Number(Heading).subtract(90.0)
theta_iRad = image.select('angle').multiply(np.pi / 180)  # 地面入射角度转为弧度
phi_iRad = ee.Image.constant(s1_azimuth_across).multiply(np.pi / 180)  # 方位角转弧度

def slop_aspect(elevation,proj,geom):
    alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
        np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡度(与地面夹角)
    phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
        np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡向角，(坡度陡峭度)坡与正北方向夹角(陡峭度)，从正北方向起算，顺时针计算角度
    phi_rRad = phi_iRad.subtract(phi_sRad)     # (飞行方向角度-坡度陡峭度)飞行方向与坡向之间的夹角
    # 分解坡度，在水平方向和垂直方向进行分解，为固定公式，cos对应水平分解，sin对应垂直分解
    alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()     # 距离向分解
    alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()     # 方位向分解
    return alpha_sRad,phi_sRad,alpha_rRad,alpha_azRad

height = DEMNASA.setDefaultProjection(proj).clip(geom)
alpha_sRad,phi_sRad,alpha_rRad,alpha_azRad = slop_aspect(DEMNASA,proj,geom)
# ------------------------------RS几何畸变区域--------------------------------- 同戴可人：高山峡谷区滑坡灾害隐患InSAR早期识别
layover = alpha_rRad.gt(theta_iRad).rename('layover')
ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
shadow = alpha_rRad.lt(ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')

def rgbmask(layover, shadow):
    r = ee.Image.constant(0).select([0], ['red'])
    g = ee.Image.constant(0).select([0], ['green'])
    b = ee.Image.constant(0).select([0], ['blue'])

    r = r.where(layover, 255)
    g = g.where(shadow, 255)

    return ee.Image.cat([r, g, b]).byte().updateMask(image.mask())

# combine layover and shadow,因为shadow和layover都是0
no_data_mask = layover.And(shadow).rename('no_data_mask')
no_data_maskrgb = rgbmask(layover, shadow)

image2 = (Eq_pixels(image.select('VV')).rename('VV_sigma0')
          .addBands(Eq_pixels(image.select('VH')).rename('VH_sigma0'))
          .addBands(Eq_pixels(image.select('angle')).rename('incAngle'))
          .addBands(Eq_pixels(layover).rename('layover'))
          .addBands(Eq_pixels(shadow).rename('shadow'))
          .addBands(Eq_pixels(no_data_mask).rename('no_data_mask'))
          .addBands(no_data_maskrgb)
          .addBands(Eq_pixels(height).rename('height')))

cal_image = (image2.addBands(ee.Image.pixelCoordinates(proj)).addBands(ee.Image.pixelLonLat())
             .reproject(crs=proj, scale=scale).updateMask(image2.select('VV_sigma0').mask()).clip(AOI_buffer))

In [24]:
model = 'volume'
sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))
gamma0 = sigma0Pow.divide(theta_iRad.cos())                                                                # 根据角度修订入射值
gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(['VV', 'VH'], ['VV_gamma0', 'VH_gamma0']) # 根据角度修订入射值
ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(gamma0dB.select('VH_gamma0')).rename('ratio_gamma0')) # gamma极化相减

def volumetric(model,theta_iRad, alpha_rRad,alpha_azRad):
    if model == 'volume':
        scf = volumetric_model_SCF(theta_iRad, alpha_rRad)
    if model == 'surface':
        scf = surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

    gamma0_flat = gamma0.divide(scf)
    gamma0_flatDB = (ee.Image.constant(10).multiply(gamma0_flat.log10()).select(['VV', 'VH'],['VV_gamma0flat', 'VH_gamma0flat']))
    ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(
        gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))
    return scf,gamma0_flat,gamma0_flatDB,ratio_flat
if orbitProperties_pass == 'ASCENDING':
    volumetric_dict['ASCENDING'] = volumetric(model,theta_iRad, alpha_rRad,alpha_azRad)
elif orbitProperties_pass == 'DESCENDING':
    volumetric_dict['DESCENDING'] = volumetric(model,theta_iRad, alpha_rRad,alpha_azRad)

In [29]:
testCollection = ee.ImageCollection([volumetric_dict['ASCENDING'][2],volumetric_dict['DESCENDING'][2]])

In [40]:
testImage_mean = testCollection.mean()
testImage_std = testCollection.reduce(ee.Reducer.stdDev())
testImage_max = testCollection.max()
testImage_min = testCollection.min()

In [41]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=8)
Map.addLayer(volumetric_dict['ASCENDING'][2].select('VV_gamma0flat'),{'min':-18,'max':5},'ASCENDING')
Map.addLayer(volumetric_dict['DESCENDING'][2].select('VV_gamma0flat'),{'min':-18,'max':5},'DESCENDING')
Map.addLayer(testImage_mean.select('VV_gamma0flat'),{'min':-18,'max':5},'mean')
Map.addLayer(testImage_max.select('VV_gamma0flat'),{'min':-18,'max':5},'max')
Map.addLayer(testImage_std.select('VV_gamma0flat'),{'min':-18,'max':5},'std')
Map.addLayer(testImage_min.select('VV_gamma0flat'),{'min':-18,'max':5},'min')
Map

EEException: Image.select: Pattern 'VV_gamma0flat' did not match any bands.